In [1]:
!pip install lightgbm imbalanced-learn joblib


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

# File path from your Google Drive
file_path = '/content/drive/MyDrive/accepted_2007_to_2018Q4.csv.gz'

# Load the compressed CSV
df = pd.read_csv(file_path, compression='gzip', low_memory=False)
df


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260696,88985880,NaN,40000.0,40000.0,40000.0,60 months,10.49,859.56,B,B3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2260697,88224441,NaN,24000.0,24000.0,24000.0,60 months,14.49,564.56,C,C4,...,NaN,NaN,Cash,Y,Mar-2019,ACTIVE,Mar-2019,10000.0,44.82,1.0
2260698,88215728,NaN,14000.0,14000.0,14000.0,60 months,14.49,329.33,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2260699,Total amount funded in policy code 1: 1465324575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Columns to use
selected_columns = [
    'loan_amnt', 'term', 'int_rate', 'grade', 'emp_length', 'home_ownership',
    'annual_inc', 'purpose', 'dti', 'delinq_2yrs', 'inq_last_6mths',
    'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
    'application_type', 'loan_status'
]

df = df[selected_columns]

# Filter relevant statuses
df = df[df['loan_status'].isin(['Fully Paid', 'Charged Off', 'Default'])]

# Create binary target
df['loan_default'] = df['loan_status'].apply(lambda x: 1 if x in ['Charged Off', 'Default'] else 0)
df.drop('loan_status', axis=1, inplace=True)


In [5]:
df

,loan_amnt,term,int_rate,grade,emp_length,home_ownership,annual_inc,purpose,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,application_type,loan_default
0,3600.0,36 months,13.99,C,10+ years,MORTGAGE,55000.0,debt_consolidation,5.91,0.0,1.0,7.0,0.0,2765.0,29.7,13.0,Individual,0
1,24700.0,36 months,11.99,C,10+ years,MORTGAGE,65000.0,small_business,16.06,1.0,4.0,22.0,0.0,21470.0,19.2,38.0,Individual,0
2,20000.0,60 months,10.78,B,10+ years,MORTGAGE,63000.0,home_improvement,10.78,0.0,0.0,6.0,0.0,7869.0,56.2,18.0,Joint App,0
4,10400.0,60 months,22.45,F,3 years,MORTGAGE,104433.0,major_purchase,25.37,1.0,3.0,12.0,0.0,21929.0,64.5,35.0,Individual,0
5,11950.0,36 months,13.44,C,4 years,RENT,34000.0,debt_consolidation,10.20,0.0,0.0,5.0,0.0,8822.0,68.4,6.0,Individual,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260688,18000.0,60 months,9.49,B,5 years,OWN,130000.0,home_improvement,20.59,0.0,1.0,17.0,0.0,23833.0,34.0,39.0,Individual,0
2260690,29400.0,60 months,13.99,C,9 years,MORTGAGE,180792.0,debt_consolidation,22.03,0.0,1.0,16.0,0.0,77480.0,85.2,32.0,Individual,0
2260691,32000.0,60 months,14.49,C,3 years,MORTGAGE,157000.0,home_improvement,10.34,0.0,0.0,14.0,0.0,111598.0,27.4,18.0,Individual,1
2260692,16000.0,60 months,12.79,C,10+ years,RENT,150000.0,medical,12.25,0.0,0.0,12.0,4.0,7700.0,55.0,28.0,Individual,0


In [6]:
# Drop columns with excessive missing data
df = df.dropna(thresh=len(df)*0.7, axis=1)

# Fill missing values
for col in df.select_dtypes(include='object'):
    df[col] = df[col].fillna(df[col].mode()[0])

for col in df.select_dtypes(include='number'):
    df[col] = df[col].fillna(df[col].median())


In [7]:
df = pd.get_dummies(df, drop_first=True)

X = df.drop('loan_default', axis=1)
y = df['loan_default']


In [8]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)


In [9]:
import lightgbm as lgb

model = lgb.LGBMClassifier(random_state=42)
model.fit(X_train_res, y_train_res)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 861401, number of negative: 861401
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.660120 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2877
[LightGBM] [Info] Number of data points in the train set: 1722802, number of used features: 47
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


LGBMClassifier(random_state=42)

In [10]:
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score

y_pred = model.predict(X_test)

print("Classification Report:\n", classification_report(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.99      0.89    215350
           1       0.54      0.07      0.13     53720

    accuracy                           0.80    269070
   macro avg       0.68      0.53      0.51    269070
weighted avg       0.76      0.80      0.74    269070

Precision: 0.5409975841978115
Recall: 0.070867460908414
F1 Score: 0.1253188932962457


In [11]:
import joblib

joblib.dump(model, 'loan_model.pkl')
joblib.dump(X.columns.tolist(), 'model_columns.pkl')


['model_columns.pkl']

In [13]:
X_test

,loan_amnt,int_rate,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,...,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,application_type_Joint App
189832,11925.0,6.24,47819.0,22.19,0.0,0.0,12.0,0.0,11935.0,35.8,...,False,False,False,False,False,False,False,False,False,False
999905,25000.0,11.99,130000.0,16.44,0.0,1.0,8.0,0.0,15017.0,23.9,...,False,False,False,False,False,False,False,False,False,False
1114019,12000.0,8.49,59000.0,14.83,0.0,0.0,8.0,0.0,13701.0,63.7,...,False,False,False,False,False,False,False,False,False,False
454706,3000.0,7.96,115000.0,22.70,0.0,0.0,18.0,0.0,42938.0,50.5,...,False,False,False,False,False,False,False,False,False,False
95243,8000.0,9.99,115000.0,13.26,1.0,0.0,13.0,0.0,20537.0,69.9,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2238397,15000.0,8.24,45000.0,27.52,0.0,0.0,21.0,0.0,10179.0,15.9,...,False,False,False,False,False,False,False,False,False,False
1218245,27000.0,6.03,72000.0,23.48,0.0,2.0,8.0,0.0,6017.0,18.1,...,False,False,False,False,False,False,False,False,False,False
1916335,30000.0,18.75,100000.0,22.70,0.0,2.0,12.0,0.0,50783.0,46.7,...,False,False,False,False,False,False,False,False,False,False
1086390,3100.0,9.17,28000.0,10.07,1.0,0.0,9.0,0.0,3908.0,28.9,...,False,False,False,False,False,False,False,False,False,False


In [18]:
X_test.to_csv('test_df.csv', index=False, encoding='utf-8')

In [12]:
from google.colab import files

files.download('loan_model.pkl')
files.download('model_columns.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
files.download('test_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>